# Text detection using Amazon Rekognition

This notebook provides a walkthrough of the [text detection API](https://docs.aws.amazon.com/rekognition/latest/dg/text-detection.html) in Amazon Rekognition. You can quickly identify text in your video and image libraries to catalog footage and photos for marketing, advertising, and media industry use cases.

In [ ]:
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [ ]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

In [ ]:
rekognition = boto3.client("rekognition")
s3 = boto3.client("s3")

In [ ]:
!mkdir -p ./tmp
temp_folder = "tmp/"

# Detect text in image

In [ ]:
imageName = "content-moderation/media/coffee.jpg"

In [ ]:
display(IImage(url=s3.generate_presigned_url("get_object", Params={"Bucket": bucket, "Key": imageName})))

# Call Rekognition to Detect Text in the Image
https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectText.html


In [ ]:
detectTextResponse = rekognition.detect_text(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name": imageName,
        }
    },
    Filters={"WordFilter": {"MinConfidence": 90}},
)

# Review the raw JSON reponse from Rekognition
In the JSON response below, you will see detected text, confidence score, and additional information.

In [ ]:
display(detectTextResponse)

# Display list of detected unsafe text

In [ ]:
import string

unsafeWords = ["crap", "darn", "damm"]
for textDetection in detectTextResponse["TextDetections"]:
    # strip punctuation before checking match
    text = textDetection["DetectedText"].translate(str.maketrans("", "", string.punctuation))
    if textDetection["Type"] == "WORD" and text in unsafeWords:
        print("Detected unsafe word: {}".format(textDetection["DetectedText"]))

# Show Bounding Boxes Around Detected Objects

In [ ]:
def drawBoundingBoxes(sourceImage, boxes):
    # blue, green, red, grey
    colors = ((255, 255, 255), (255, 255, 255), (76, 182, 252), (52, 194, 123))

    # Download image locally
    imageLocation = temp_folder + os.path.basename(sourceImage)
    s3.download_file(bucket, sourceImage, imageLocation)

    # Draws BB on Image
    bbImage = Image.open(imageLocation)
    draw = ImageDraw.Draw(bbImage)
    width, height = bbImage.size
    col = 0
    maxcol = len(colors)
    line = 3
    for box in boxes:
        x1 = int(box[1]["Left"] * width)
        y1 = int(box[1]["Top"] * height)
        x2 = int(box[1]["Left"] * width + box[1]["Width"] * width)
        y2 = int(box[1]["Top"] * height + box[1]["Height"] * height)

        draw.text((x1, y1), box[0], colors[col])
        for l in range(line):
            draw.rectangle((x1 - l, y1 - l, x2 + l, y2 + l), outline=colors[col])
        col = (col + 1) % maxcol

    imageFormat = "PNG"
    ext = sourceImage.lower()
    if ext.endswith("jpg") or ext.endswith("jpeg"):
        imageFormat = "JPEG"

    bbImage.save(imageLocation, format=imageFormat)

    display(bbImage)

In [ ]:
boxes = []
textDetections = detectTextResponse["TextDetections"]
for textDetection in textDetections:
    boxes.append((textDetection["Type"], textDetection["Geometry"]["BoundingBox"]))

drawBoundingBoxes(imageName, boxes)

# Detect Text in Image using Filters and Regions of Interest

In [ ]:
imageName = "content-moderation/media/coffee.jpg"

In [ ]:
display(IImage(url=s3.generate_presigned_url("get_object", Params={"Bucket": bucket, "Key": imageName})))

In [ ]:
# Call Amazon Rekognition to detect text in the image
# https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectText.html

detectTextResponse = rekognition.detect_text(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name": imageName,
        }
    },
    Filters={
        "WordFilter": {"MinConfidence": 90, "MinBoundingBoxHeight": 0.05, "MinBoundingBoxWidth": 0.02},
        "RegionsOfInterest": [
            {"BoundingBox": {"Width": 0.1, "Height": 0.05, "Left": 0.01, "Top": 0.01}},
        ],
    },
)

# Review JSON Response from Rekognition Text API (Text Detection)
In the JSON response below, you will see detected text, confidence score, and additional information.


In [ ]:
display(detectTextResponse)

In [ ]:
for textDetection in detectTextResponse["TextDetections"]:
    text = textDetection["DetectedText"]
    if textDetection["Type"] == "WORD":
        print("Word: {}".format(textDetection["DetectedText"]))

# Detect Text in Video
 Text detection in video is an async operation. 
https://docs.aws.amazon.com/rekognition/latest/dg/text-detecting-video-procedure.html.

- First we start a text detection job which returns a Job Id.
- We can then call `get_text_detection` to get the job status and after job is complete, we can get object metadata.
- In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.

In [ ]:
videoName = "content-moderation/media/serverless-bytes.mov"

strDetail = "Text detected in video<br>=======================================<br>"
strOverall = "Text in the overall video:<br>=======================================<br>"

In [ ]:
s3VideoUrl = s3.generate_presigned_url("get_object", Params={"Bucket": bucket, "Key": videoName})

videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(
    s3VideoUrl
)

videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)

display(HTML(videoui))

# Call Rekognition to start a job for text detection

In [ ]:
startTextDetection = rekognition.start_text_detection(
    Video={
        "S3Object": {
            "Bucket": bucket,
            "Name": videoName,
        }
    },
)

textJobId = startTextDetection["JobId"]
display("Job Id: {0}".format(textJobId))

# Wait for Text Detection Job to Complete
In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.


In [ ]:
getTextDetection = rekognition.get_text_detection(JobId=textJobId)

while getTextDetection["JobStatus"] == "IN_PROGRESS":
    time.sleep(5)
    print(".", end="")

    getTextDetection = rekognition.get_text_detection(JobId=textJobId)

display(getTextDetection["JobStatus"])

# Review Raw JSON Response from Rekognition Text Detection API
In the JSON response below, you will see list of detected text.

For each detected object, you will see information like `Timestamp`.

In [ ]:
display(getTextDetection)

# Display Recognized Text in the Video

In [ ]:
flaggedTextInVideo = ["AWS", "Twitter"]

theLines = {}

# Objects detected in each frame
for obj in getTextDetection["TextDetections"]:
    if obj["TextDetection"]["Type"] == "WORD":
        ts = obj["Timestamp"]
        cconfidence = obj["TextDetection"]["Confidence"]
        oname = obj["TextDetection"]["DetectedText"]

        if oname in flaggedTextInVideo:
            print("Found flagged text at {} ms: {} (Confidence: {})".format(ts, oname, round(cconfidence, 2)))

        strDetail = strDetail + "At {} ms: {} (Confidence: {})<br>".format(ts, oname, round(cconfidence, 2))
        if oname in theLines:
            cojb = theLines[oname]
            theLines[oname] = {"Text": oname, "Count": 1 + cojb["Count"]}
        else:
            theLines[oname] = {"Text": oname, "Count": 1}

# Unique objects detected in video
for theLine in theLines:
    strOverall = strOverall + "Name: {}, Count: {}<br>".format(theLine, theLines[theLine]["Count"])

# Display results
display(HTML(strOverall))

In [ ]:
listui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(strDetail)
display(HTML(listui))

# References
- https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectText.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_StartTextDetection.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetTextDetection.html

# Congratulations!
You have successfully used Amazon Rekognition to identify text in images an videos.

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}